In [29]:
import os
import csv
import math
import itertools
import pickle

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve, train_test_split, KFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [30]:
df = pd.read_csv("./data/adult_processed.csv")
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,72,0,2649,11,8,0,0,1,4,1,0,91,39,39,0
1,65,4,6514,11,8,0,4,1,4,1,0,91,17,39,0
2,49,0,11175,15,9,0,0,4,2,1,0,91,39,39,0
3,37,4,7009,5,3,0,7,4,4,1,0,90,39,39,0
4,24,4,16850,15,9,0,10,3,4,1,0,90,39,39,0


In [31]:
# extracted from https://www.kaggle.com/code/prashant111/eda-logistic-regression-pca#Introduction-to-Principal-Component-Analysis-(PCA)

X = df.drop(['income'], axis=1)
y = df['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)
X_test.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,1.273909,0.086059,1.117381,-1.107252,-1.980744,-0.950671,1.293604,1.591474,0.39298,1.430470,-0.279828,-0.2046,-0.034187,0.292864
1,-1.436966,0.086059,0.824846,0.184396,-0.423425,-0.950671,-0.126801,0.968753,0.39298,1.430470,-0.279828,-0.2046,-1.272158,0.292864
2,-1.143899,0.086059,-0.742001,1.217715,-0.034095,-0.950671,0.346667,-0.276689,-3.14167,1.430470,-0.279828,-0.2046,-0.446844,0.420259
3,-0.118162,-1.962098,-1.493974,-0.332263,1.133894,1.051889,0.820136,-0.899410,0.39298,-0.699071,-0.279828,-0.2046,-0.034187,0.292864
4,0.761041,0.086059,1.713512,0.442726,1.523223,1.051889,1.293604,-0.899410,0.39298,-0.699071,-0.279828,-0.2046,0.791128,0.292864


In [32]:
pca= PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
dim = np.argmax(cumsum >= 0.90) + 1
print('The number of dimensions required to preserve 90% of variance is',dim)

The number of dimensions required to preserve 90% of variance is 12


In [33]:
X = df.drop(['income','native.country', 'hours.per.week'], axis=1)
y = df['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X.columns)

X_test = pd.DataFrame(scaler.transform(X_test), columns = X.columns)
X_test.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss
0,1.273909,0.086059,1.117381,-1.107252,-1.980744,-0.950671,1.293604,1.591474,0.39298,1.430470,-0.279828,-0.2046
1,-1.436966,0.086059,0.824846,0.184396,-0.423425,-0.950671,-0.126801,0.968753,0.39298,1.430470,-0.279828,-0.2046
2,-1.143899,0.086059,-0.742001,1.217715,-0.034095,-0.950671,0.346667,-0.276689,-3.14167,1.430470,-0.279828,-0.2046
3,-0.118162,-1.962098,-1.493974,-0.332263,1.133894,1.051889,0.820136,-0.899410,0.39298,-0.699071,-0.279828,-0.2046
4,0.761041,0.086059,1.713512,0.442726,1.523223,1.051889,1.293604,-0.899410,0.39298,-0.699071,-0.279828,-0.2046


In [34]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print('Logistic Regression accuracy score with the first 12 features: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Logistic Regression accuracy score with the first 12 features: 0.8348


In [35]:
# Save the model to a file
with open('./model/adult_LR.pkl', 'wb') as f:
    pickle.dump(logreg, f)